In [1]:
USERNAME = 'OOIAPI-T4K9H80QN79JPD'
TOKEN =  '2QXI1W549EC1EU'

# import the python packages
import requests
import datetime
import matplotlib.pyplot as plt
import numpy as np
from datetime import date
import calendar

SENSOR_BASE_URL = 'https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/'

r = requests.get(SENSOR_BASE_URL, auth=(USERNAME, TOKEN))

In [2]:
# function to make an API request and print the results
def get_and_print_api(url):
  r = requests.get(url, auth=(USERNAME, TOKEN))
  data = r.json()
  for d in data:
    print( d )


In [ ]:
def ntp_seconds_to_datetime(ntp_seconds):
   return datetime.datetime.utcfromtimestamp(ntp_seconds - ntp_delta).replace(microsecond=0)


In [ ]:
temp_binned = np.zeros((510, 51))
salt_binned = np.zeros((510, 51))

kk=-1

for i in range(2014, 2019, 1):
    
    if i == 2014:
        mm=10
    else:
        mm=1
    
    for j in range(mm, 13, 1):
        
        if i == 2018 and j >= 3:
            # Instrument Information
            site = 'CE09OSPM'
            node = 'WFP01'
            instrument = '03-CTDPFK000'
            method = 'telemetered'
            stream = 'ctdpf_ckl_wfp_instrument'
        else:
            # Instrument Information
            site = 'CE09OSPM'
            node = 'WFP01'
            instrument = '03-CTDPFK000'
            method = 'recovered_wfp'
            stream = 'ctdpf_ckl_wfp_instrument_recovered'
    
        start_date = date(i,j,1)
        days_in_month = calendar.monthrange(start_date.year, start_date.month)[1]
        
        if j < 10:
            month='0'+str(j)
        else:
            month=str(j)
        
        params = {
          'beginDT':str(i)+'-'+month+'-'+str(days_in_month)+'T00:00:00.000Z',
          'endDT':str(i)+'-'+month+'-'+str(days_in_month)+'T23:59:59.000Z',
          'limit':20000,   
        }
        
        data_request_url ='/'.join((SENSOR_BASE_URL,site,node,instrument,method,stream))
        r = requests.get(data_request_url, params=params, auth=(USERNAME, TOKEN))
        data = r.json()
        
        if len(data) == 2:
           print('no data for '+str(i)+'-'+month)
           kk=kk+1
        else:
           print('found some data for '+str(i)+'-'+month)
           # Time Processing Routine
           ntp_epoch = datetime.datetime(1900, 1, 1)
           unix_epoch = datetime.datetime(1970, 1, 1)
           ntp_delta = (unix_epoch - ntp_epoch).total_seconds()
        
           # Process the data
           time = []
           pressure = []
           temperature = []
           salinity = []
           for xx in range(len(data)):
             time.append(ntp_seconds_to_datetime(data[xx]['time']))
             pressure.append(data[xx]['ctdpf_ckl_seawater_pressure'])
             temperature.append(data[xx]['ctdpf_ckl_seawater_temperature'])
             salinity.append(data[xx]['practical_salinity'])
        
           temperature=np.array(temperature)
           pressure=np.array(pressure)
           salinity=np.array(salinity)
        
           bins = np.arange(511)   #1m bins; at 25cm/s ~ 4obs/1m bin
            
           temp_array = np.zeros((len(bins)-1,))
           temp_array[temp_array == 0.0] = np.nan
           salt_array = np.zeros((len(bins)-1,))
           salt_array[salt_array == 0.0] = np.nan
           
           k=-1
           for ii in range(len(bins)-1):
               ind = np.where((pressure >= bins[ii]) & (pressure < bins[ii+1]))
               k=k+1
               temp_array[k] = np.nanmean(temperature[ind])
               salt_array[k] = np.nanmean(salinity[ind])

           kk=kk+1
           temp_binned[:,kk]=temp_array
           salt_binned[:,kk]=salt_array

